In [ ]:
import plot_SNVM_data as pSd
import SNVM_B_xyz_extraction as Sdp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib_scalebar.scalebar import ScaleBar
from math import ceil,floor,pi
import colorsys
import cv2
from skimage.util.shape import view_as_blocks
from skimage.io import imsave
from skimage.measure import profile_line
import scipy.ndimage

In [ ]:
basepath = '/home/julisko/Seafile/My Library/SNVM/20230227_QZabre_Visit/Visit_Feb_2023'
folder = '/2023-03-04/odmr_fast' #DyIG circle
folderday = '/2023-03-04-17-16-26-odmr_fast'
#folder = '/2023-02-27/odmr_fast' #Fe2O3 Stripes
#folderday = '/2023-02-27-15-02-13-odmr_fast'
#folder = '/2023-02-27/odmr_fast' #Fe2O3 Stripes super high res overnight scan
#folderday = '/2023-02-27-18-58-59-odmr_fast'

In [ ]:
# ReadSNVMfile (filename) -> red file into file, metadata
# ExtractData (file, datatype = 'odmr:mTesla') -> Extract datatype from file
# AdjustDataToQS3 (data) -> adjust directions to fit to QS3 plotting
# PlotData(data, metadata, minval=None,maxval=None,datalabel='',zscaling=1,backgroundcorr=None)
#   -> Plot data, backgroundcorr subtracts mean value and sets min to zero, possible values: 'x', 'y' and None
# MeanCorrY(data) -> subtract mean of data for every y line
# MeanCorrX(data) -> analogous for x dir
# SetMinZero(data) -> subtract minimum from data

In [ ]:
# expand_Data(data, nr_expansions=1) -> returns data put next to each other in a square of nr_expansions by nr_expansions
# apply_Hann_window(data, metadata,nr_expansions=1,return_freq=False) -> apply 2D Hanning window to data, return_freq additionally returns frequency axis (data is return[0], freq is return[1])
# get_kvectors1D(data, metadata) -> returns 1D k-vectors and additional infos (kx,ky,px_y,px_y,dx,dy) (px is pixel size and dx or dy is length per pixel) based on pixel and actual size of data
# get_Kvectors2D(data, metadata, eps = 0.001) -> returns 2D K-vectors as (Kx,Ky,K) with added eps(ilon) to avoid division by zero
# get_eta(K,dz=0) -> returns eta (see Degen script) with additional z-offset dz
# get_NV_unit_vectors(theta,phi) -> returns unit vectors of NV center (projection between r,theta,phi and x,y,z) as (ex,ey,ez)
# transform_BFT_to_components(Bft,Kx,Ky,K,eta,ex,ey,ez) -> returns stray field components of K-space BFT as (BxFT,ByFT,BzFT,KNV)
# crop_expansion(data_expanded,nr_expansions=1) -> crops images transformed back into real space to original size, depending on nr_expansions
# project_BIP_onto_angle(Bx,By,proj_ang) -> returns stray field projected into IP direction along angle proj_ang 

In [ ]:
file,metadata = pSd.ReadSNVMfile(basepath+folder+folderday)

In [ ]:
data = pSd.ExtractData(file,datatype = 'odmr:mTesla')
data = pSd.AdjustDataToQS3(data)
pSd.PlotData(data, metadata, datalabel='Stray Field')#,minval=0.55,maxval=0.8)#,backgroundcorr='y')
plt.show()

In [ ]:
#topo = pSd.ExtractData(file,datatype = 'afm:height')
#topo = pSd.AdjustDataToQS3(topo)
#pSd.PlotData(topo,metadata, backgroundcorr = None)
#plt.show()

In [ ]:
#expand data (copies next to each other) for better (?) FFT
nr_expansions=1
data_expanded = Sdp.expand_Data(data,nr_expansions=nr_expansions)
pSd.PlotData(data_expanded,metadata)
plt.show()

#nr_expansions = 1
#data_expanded = nr_expansions*[data]
#data_expanded = nr_expansions*[data_expanded]
#data_expanded = np.block(data_expanded)
#data_expanded=data

In [ ]:
Bft = np.fft.fftshift(np.fft.fft2(data_expanded))
pSd.PlotData(np.log(np.abs(Bft)),metadata,datalabel = 'FFT Amp')#,maxval=1000)
plt.show()

In [ ]:
#size = metadata['rect']['size']
#resolution = metadata['resolution']
#freq_x = np.fft.fftfreq(nr_expansions*resolution[0],d=size[0]/nr_expansions*resolution[0])
#freq_y = np.fft.fftfreq(nr_expansions*resolution[1],d=size[1]/nr_expansions*resolution[1])
#print(size, resolution, freq_x, np.shape(data))
#window_x = np.hanning(freq_x.size)
#window_y = np.hanning(freq_y.size)
#window_2d = np.sqrt(np.outer(window_x,window_y))
#print(freq_x.size,freq_y.size,window_x.size,window_y.size,np.shape(window_2d))
#Bft_hann = window_2d.transpose()*Bft

Bft_hann = Sdp.apply_Hann_window(Bft,metadata,nr_expansions=nr_expansions)
pSd.PlotData(np.log(np.abs(Bft_hann)),metadata)#,maxval = 100)
plt.show()

In [ ]:
#pSd.PlotData(np.abs(np.fft.ifft2(np.fft.ifftshift(Bft))),metadata)
#plt.show()
#pSd.PlotData(np.abs(np.fft.ifft2(np.fft.ifftshift(Bft_hann))),metadata)
#plt.show()

In [ ]:
#define 1D k vectors
kx, ky, px_x, px_y, dx, dy = Sdp.get_kvectors1D(Bft_hann,metadata)

#px_x = np.shape(Bft_hann)[0]
#px_y = np.shape(Bft_hann)[1]
#actualshape = metadata['rect']['size'] #get shape in m
#print(actualshape[0]/px_x)
#dx = actualshape[0]/px_x #length per pixel in x dir realspace
#dy = actualshape[1]/px_y #length per pixel in y dir realspace
#if np.mod(px_x,2)==0:
#    kx = 2*np.pi*np.linspace(-px_x/2,px_x/2-1,px_x-1)/(dx*px_x)
#    # print(kx,np.shape(kx))
#else: 
#    kx = 2*np.pi*np.linspace(-(px_x-1)/2,(px_x-1)/2,px_x)/(dx*px_x)
#    # print(kx,np.shape(kx))

#if np.mod(px_y,2)==0:
#    ky = 2*np.pi*np.linspace(-px_y/2,px_y/2-1,px_y-1)/(dy*px_y)
#    # print(ky,np.shape(ky))
#else: 
#    ky = 2*np.pi*np.linspace(-(px_y-1)/2,(px_y-1)/2,px_y)/(dy*px_y)
#    # print(ky,np.shape(ky))

In [ ]:
#define 2D k vectors
#Kx, Ky, K = Sdp.get_Kvectors2D(Bft_hann,metadata)
Kx, Ky, K = Sdp.get_Kvectors2D(Bft,metadata)

#eps = 0.001 # add to avoid division by zero
#Kx = np.outer(np.ones(px_y-1),kx) + eps
#Ky = np.outer(ky,np.ones(px_x-1)) + eps
#K = np.sqrt(Kx**2+Ky**2)

In [ ]:
#dz=80e-9 # optional shift in z
#eta = np.exp(-K*dz)
dz=0*200e-9
eta = Sdp.get_eta(K,dz=dz)

In [ ]:
# define probe parameters B-G15-18 for Fe2O3 image below box 
theta = 144.75
phi = 270

# probe params for B-I09-17 used to circle with 'ears' 
#theta = 90
#phi = 35.25

ex,ey,ez = Sdp.get_NV_unit_vectors(theta,phi)

#ex = np.sin(theta)*np.cos(phi)
#ey = np.sin(theta)*np.sin(phi)
#ez = np.cos(theta)
#print(ex,ey,ez)

In [ ]:
# transform Bft to components 
#Bxft, Byft, Bzft, KNV = Sdp.transform_BFT_to_components(Bft_hann,Kx,Ky,K,
#                                                        eta,ex,ey,ez)
Bxft, Byft, Bzft, KNV = Sdp.transform_BFT_to_components(Bft,Kx,Ky,K,
                                                        eta,ex,ey,ez)

#KNV = (1j*ex*Kx + 1j*ey*Ky - ez*K)
#Bxft = 1j*(Kx/KNV)*Bft[:-1,:-1]*eta
#Byft = 1j*(Ky/KNV)*Bft[:-1,:-1]*eta
#Bzft = -(K/KNV)*Bft[:-1,:-1]*eta

In [ ]:
#test plots of FFT of B components
#pSd.PlotData(np.real(Bxft),metadata,maxval=100)
#plt.show()
#pSd.PlotData(np.real(Byft),metadata,maxval=100)
#plt.show()
#pSd.PlotData(np.real(Bzft),metadata,maxval=100)
#plt.show()

In [ ]:
# inverse fft of B components
Bx_expanded = np.real(np.fft.ifft2(np.fft.fftshift(Bxft)))
By_expanded = np.real(np.fft.ifft2(np.fft.fftshift(Byft)))
Bz_expanded = np.real(np.fft.ifft2(np.fft.fftshift(Bzft)))
#pSd.PlotData(Bx_expanded,metadata)
#plt.show()
#pSd.PlotData(By_expanded,metadata)
#plt.show()
#pSd.PlotData(Bz_expanded,metadata)
#plt.show()

In [ ]:
#crop images back after inverse FFT 
Bx = Sdp.crop_expansion(Bx_expanded,nr_expansions=nr_expansions)
By = Sdp.crop_expansion(By_expanded,nr_expansions=nr_expansions)
Bz = Sdp.crop_expansion(Bz_expanded,nr_expansions=nr_expansions)

#Bx = Bx[:int(np.shape(Bx)[0]/nr_expansions),:int(np.shape(Bx)[1]/nr_expansions)]
#By = By[:int(np.shape(By)[0]/nr_expansions),:int(np.shape(By)[1]/nr_expansions)]
#Bz = Bz[:int(np.shape(Bz)[0]/nr_expansions),:int(np.shape(Bz)[1]/nr_expansions)]

In [ ]:
pSd.PlotData(Bz-np.mean(Bz),metadata,datalabel=r'B$_{OOP}$',cmap='bwr',maxval=2.1e-1,minval=-2.1e-1)
plt.show()

In [ ]:
#for proj_ang in np.arange(0,360,10):
#    B_proj = Sdp.project_BIP_onto_angle(Bx,By,proj_ang)
    #pSd.PlotData(B_proj-np.mean(B_proj),metadata,datalabel=str(proj_ang)+'°',cmap='bwr')
    #plt.show()
    
    #proj_ang_rad = proj_ang*2*np.pi/360
    #B_IP = np.sqrt(Bx**2+By**2)
    #angle_B_IP = np.arctan(Bx/By)
    #angle_B_proj_B_IP = proj_ang_rad-angle_B_IP
    #B_proj = np.sin(angle_B_proj_B_IP)*B_IP
    #print(proj_ang_rad, B_IP, angle_B_IP, angle_B_proj_B_IP)
    #pSd.PlotData(B_proj-np.mean(B_proj),metadata,datalabel=str(proj_ang)+'°',cmap='bwr')
    #plt.show()

In [ ]:
proj_ang = -25
B_proj = Sdp.project_BIP_onto_angle(Bx,By,proj_ang)
B_proj_QS3 = pSd.AdjustDataToQS3(B_proj)
pSd.PlotData(B_proj-np.mean(B_proj),metadata,datalabel=str(proj_ang)+'°',cmap='bwr_r',maxval=0.021,minval=-0.021)
plt.show()

#proj_ang = 10
#B_proj = Sdp.project_BIP_onto_angle(Bx,By,proj_ang)
#proj_ang_rad = proj_ang *2*np.pi/360
#B_IP = np.sqrt(Bx**2+By**2)
#angle_B_IP = np.arctan(Bx/By)
#angle_B_proj_B_IP = proj_ang_rad-angle_B_IP
#B_proj = np.sin(angle_B_proj_B_IP)*B_IP
#B_proj = np.sin(proj_ang_rad)*B_IP
#print(angle_B_IP,angle_B_proj_B_IP,B_proj)
#pSd.PlotData(B_proj-np.mean(B_proj),metadata,datalabel=str(proj_ang)+'°',cmap='bwr_r')#,maxval=0.015,minval=-0.015)
#plt.show()


In [ ]:
# plot B_IP projected onto angles in list proj_angles

proj_angles = np.arange(0,360,10)
B_proj_all = np.zeros((len(proj_angles),np.shape(Bx)[0],np.shape(Bx)[1])) 

fig = plt.figure(figsize = (24,20))

for i in range(len(proj_angles)):
    proj_ang = proj_angles[i]
    B_proj = Sdp.project_BIP_onto_angle(Bx,By,proj_ang)
    B_proj_all[i,:,:] = B_proj
    ax = fig.add_subplot(ceil((np.sqrt(len(proj_angles)))),
                         ceil((np.sqrt(len(proj_angles)))),
                         i+1)
    im = ax.imshow(B_proj, cmap='bwr')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title('Projection onto ' + str(proj_ang)+ '°')

    px_shape = metadata['resolution'] #get shape in pxs
    actualshape = metadata['rect']['size'] #get shape in m
    scalebar = ScaleBar(actualshape[0]/px_shape[0], length_fraction=0.5, 
                    width_fraction=0.02, box_alpha=0.5,frameon=True,box_color='w',location='lower right')
    ax.add_artist(scalebar)

    cb=fig.colorbar(im)
    cb.set_label(r'$B_{projected}$')
        
plt.show()

#fig1,ax1 = plt.subplots(6,6,figsize = (20,20))

#for i  in range(6):
#    for j in range(6):
#        im = ax1[i][j].imshow(B_proj_all[i+j+(np.shape(ax)[0]-1)*i,:,:], cmap='bwr')
#        ax1[i][j].get_xaxis().set_visible(False)
#        ax1[i][j].get_yaxis().set_visible(False)
#        ax1[i][j].set_title('Projection onto ' + str(proj_angles[i+j+(np.shape(ax)[0]-1)*i]) + '°')
#plt.show()
    

In [ ]:
# plot B_IP for different projection angles and different theta_NV or phi_NV

proj_angles = np.arange(0,360,10)

phis = np.arange(260,284,4)
thetas = np.arange(95,165,10)

dz=100e-9

B_proj_all = np.zeros((len(proj_angles),np.shape(Bx)[0],np.shape(Bx)[1])) 

#for phi in phis:
for theta in thetas:
    
    #theta = 144.75
    theta = theta
    phi = 270
    #phi = phi
    
    Bx, By, Bz = Sdp.extract_Bxyz(data, metadata, theta, phi, eps = 0.001, nr_expansions=1, dz=dz, apply_hann=False)
    
    fig = plt.figure(figsize=(24,20))
    
    #fig, ax = plt.subplots(ceil((np.sqrt(len(proj_angles)))),ceil((np.sqrt(len(proj_angles)))),figsize = (20,20))
    #plt.suptitle(r'$\varphi_{NV}$ = ' + str(phi) + '°',fontsize = 25)
    plt.suptitle(r'$\theta_{NV}$ = ' + str(theta) + '°',fontsize = 25)
            
    for i  in range(len(proj_angles)):
        
        proj_ang = proj_angles[i]
        B_proj = Sdp.project_BIP_onto_angle(Bx,By,proj_ang)
        B_proj_all[i,:,:] = B_proj
        
        ax = fig.add_subplot(ceil((np.sqrt(len(proj_angles)))),
                             ceil((np.sqrt(len(proj_angles)))),
                             i+1)
        im = ax.imshow(B_proj, cmap='bwr')
        
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.set_title('Projection onto ' + str(proj_ang)+ '°')
        
        px_shape = metadata['resolution'] #get shape in pxs
        actualshape = metadata['rect']['size'] #get shape in m
        scalebar = ScaleBar(actualshape[0]/px_shape[0], length_fraction=0.5, 
                        width_fraction=0.02, box_alpha=0.5,frameon=True,box_color='w',location='lower right')
        ax.add_artist(scalebar)

        cb=fig.colorbar(im)
        cb.set_label(r'$B_{projected}$')
            
    plt.tight_layout()
            
plt.show()

In [ ]:
# plot of angles of B_IP

dz=100e-9
#theta = 83
#phi = 35.25
theta = 144.75
phi = 270



Bx, By, Bz = Sdp.extract_Bxyz(data, metadata, theta, phi, eps = 0.001, nr_expansions=1, dz=dz, apply_hann=False)

Bx = Bx-np.mean(Bx)
By = By-np.mean(By)
Bz = Bz-np.mean(Bz)

plt.imshow(By)
plt.title('B_y')

angle_B_IP_rad = np.arctan2(By,Bx)
angle_B_IP_deg = angle_B_IP_rad*360/(2*np.pi)

fig, ax = plt.subplots()
im = plt.imshow(angle_B_IP_deg,cmap = 'twilight')
cb=fig.colorbar(im)
cb.set_label('B_IP Angle')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

print(np.max(angle_B_IP_deg),np.min(angle_B_IP_deg),np.max(angle_B_IP_deg)-np.min(angle_B_IP_deg))

In [ ]:
# plot angle with saturation set by projected B_IP 

# proj_angles = np.arange(0,360,10)
# B_proj_all = np.zeros((len(proj_angles),np.shape(Bx)[0],np.shape(Bx)[1])) 

dz=0*200e-9
theta = 85
phi = 35.25
#theta = 144.75
#phi = 270


Bx, By, Bz = Sdp.extract_Bxyz(data, metadata, theta, phi, eps = 0.001, nr_expansions=1, dz=dz, apply_hann=False)

Bx_mean = np.mean(Bx)
By_mean = np.mean(By)
Bz_mean = np.mean(Bz)
Bx = Bx-Bx_mean
By = By-By_mean
Bz = Bz-Bz_mean


angles = np.zeros((np.shape(Bx)[0],np.shape(Bx)[1]))
B_IP_proj = np.zeros((np.shape(Bx)[0],np.shape(Bx)[1]))
angles_x_unitcircle = np.zeros((np.shape(Bx)[0],np.shape(Bx)[1]))
angles_y_unitcircle = np.zeros((np.shape(By)[0],np.shape(By)[1]))

for i in range(np.shape(Bx)[0]):
    for j in range(np.shape(Bx)[1]):
        angle_B_IP_rad = np.arctan2(By[i,j],Bx[i,j])
        angle_B_IP_deg = angle_B_IP_rad*360/(2*np.pi)
        angles[i,j] = angle_B_IP_deg 
        B_IP_proj[i,j] = np.sqrt(Bx[i,j]**2+By[i,j]**2)
        angles_x_unitcircle[i,j] = np.sin(angle_B_IP_rad)
        angles_y_unitcircle[i,j] = np.cos(angle_B_IP_rad)
        

#x = np.linspace(0,np.shape(Bx)[0]-1,np.shape(Bx)[0])
#y = np.linspace(0,np.shape(Bx)[1]-1,np.shape(Bx)[1])

print(np.shape(Bx))



Nr_arrows_per_line = 50 #268/4 = 67 bc view_as_blocks has to fit in perfectly and size of Bx of 270 doesn't work well
data_cutoff_min = 0
data_cutoff_max = 250
blocksize = int((data_cutoff_max-data_cutoff_min)/Nr_arrows_per_line) #blocksize has to be integer before conversion


x = np.linspace(data_cutoff_min,data_cutoff_max-1,Nr_arrows_per_line)
y = np.linspace(data_cutoff_min,data_cutoff_max-1,Nr_arrows_per_line)


##### prepare Bxy to be plotted as arrows ####

Bx_blocks = view_as_blocks(Bx[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max],
                           block_shape=(blocksize,blocksize))
By_blocks = view_as_blocks(By[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max],
                           block_shape=(blocksize,blocksize))

Bx_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))
By_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))

for i in range(np.shape(Bx_blocks)[0]):
    for j in range(np.shape(Bx_blocks)[1]):
        Bx_reduced_avg[i,j] = np.mean(Bx_blocks[i,j,:,:])

for i in range(np.shape(By_blocks)[0]):
    for j in range(np.shape(By_blocks)[1]):
        By_reduced_avg[i,j] = np.mean(By_blocks[i,j,:,:])

#print(np.shape(Bx_reduced_avg),np.shape(By_reduced_avg),Bx_reduced_avg,By_reduced_avg)
#print(np.shape(Bx),np.shape(x),np.shape(y),np.shape(Bx_reshaped),np.shape(By_reshaped))
#print(np.shape(Bx),x,y,np.shape(Bx_reshaped),np.shape(By_reshaped))

###############################################

fig, ax = plt.subplots()
#fig = plt.figure(figsize=(20,20))
im = ax.imshow(angles[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max],cmap = 'hsv')#,vmin=-180,vmax=100)
#arrows = ax.quiver(x,y,Bx_reduced_avg,By_reduced_avg)
cb=fig.colorbar(im)
cb.set_label('B_IP Angle')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
#imsave('/home/julisko/Seafile/My Library/SNVM/20230227_QZabre_Visit/Analysis/angles_hematite.tiff',
#       angles[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max],plugin='tifffile')

print(np.max(angles),np.min(angles))

######## try to change saturation with B_IP strength ########

#plt.show()
fig.canvas.draw()
orig_im_RGBarray = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
orig_image = orig_im_RGBarray.reshape(fig.canvas.get_width_height()[::-1] + (3,))

angle_im = cv2.cvtColor(orig_image,cv2.COLOR_BGR2HSV_FULL)
h=angle_im[:,:,0].copy()
s=angle_im[:,:,1].copy()
v=angle_im[:,:,2].copy()
s=s
#v=v+0.001
angle_im=np.dstack((h,s,v))
angle_im=cv2.cvtColor(angle_im,cv2.COLOR_HSV2RGB_FULL)

#plt.imshow(angle_im)

##############################################################


angles_x_blocks = view_as_blocks(angles_x_unitcircle[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max]
                                 ,block_shape=(blocksize,blocksize))
angles_y_blocks = view_as_blocks(angles_y_unitcircle[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max]
                                 ,block_shape=(blocksize,blocksize))
angles_blocks = view_as_blocks(angles[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max],
                               block_shape=(blocksize,blocksize))

angles_x_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))
angles_y_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))
angles_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))

for i in range(np.shape(angles_x_blocks)[0]):
    for j in range(np.shape(angles_x_blocks)[1]):
        angles_x_reduced_avg[i,j] = np.mean(angles_x_blocks[i,j,:,:])

for i in range(np.shape(angles_y_blocks)[0]):
    for j in range(np.shape(angles_y_blocks)[1]):
        angles_y_reduced_avg[i,j] = np.mean(angles_y_blocks[i,j,:,:])
        
for i in range(np.shape(angles_blocks)[0]):
    for j in range(np.shape(angles_blocks)[1]):
        angles_reduced_avg[i,j] = np.mean(angles_blocks[i,j,:,:])
        
#print(angles_x_reduced_avg,angles_y_reduced_avg)   
print(np.max(angles_reduced_avg),np.min(angles_reduced_avg))
#print(np.shape(B_IP_proj[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max]))

u = np.zeros_like(angles_x_reduced_avg)+np.sqrt(0.5)
v = np.zeros_like(angles_y_reduced_avg)+np.sqrt(0.5)

########## Plots angles after averaging #######
#fig, ax = plt.subplots()
##fig = plt.figure(figsize=(20,20))
#im = ax.imshow(angles_reduced_avg,cmap = 'twilight')#,vmin=-180,vmax=100)
##arrows = ax.quiver(x,y,Bx_reduced_avg,By_reduced_avg)
#cb=fig.colorbar(im)
#cb.set_label('B_IP Angle')
#ax.get_xaxis().set_visible(False)
#ax.get_yaxis().set_visible(False)
###############################################

B_x_onlyIP = data/np.sin(phi)
B_y_onlyIP = data/np.cos(phi)

fig, ax = plt.subplots()
im = ax.imshow(B_IP_proj[data_cutoff_min:data_cutoff_max,data_cutoff_min:data_cutoff_max],cmap = 'summer',
               extent = (data_cutoff_min,data_cutoff_max,data_cutoff_max,data_cutoff_min))
#arrows = ax.quiver(x+0.5,y+0.5,angles_x_reduced_avg,angles_y_reduced_avg,scale=50,angles=angles_reduced_avg,pivot='mid') 
arrows = ax.quiver(x+0.5,y+0.5,u,v,scale=50,angles=angles_reduced_avg,pivot='mid') 
# angles_xy not used anymore, only for length of arrows. directions are directly set by angles
cb=fig.colorbar(im)
cb.set_label('B_IP')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
        
fig, ax = plt.subplots()
im = ax.imshow(Bz,cmap = 'summer')
cb=fig.colorbar(im)
cb.set_label('B_z')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)


#proj_ang_rad = proj_ang*2*np.pi/360
#B_IP = np.sqrt(Bx**2+By**2)
#angle_B_IP = np.arctan2(By,Bx)
#angle_B_proj_B_IP = proj_ang_rad-angle_B_IP
#B_proj = np.sin(angle_B_proj_B_IP)*B_IP
#B_proj = np.sin(angle_B_proj_B_IP)*B_IP
#print(proj_ang_rad, B_IP, angle_B_IP, angle_B_proj_B_IP)

############################## Feed Back into Program #######################################

data_x = Bx+Bx_mean
data_y = By+By_mean
data_z = Bz+Bz_mean

ex, ey, ez = Sdp.get_NV_unit_vectors(theta,phi)
print(ex,ey,ez)
B_NV_test = (data_x*ex + data_y*ey  + data_z*ez) 

pSd.PlotData(B_NV_test,metadata,datalabel='B_NV_reinput')
plt.show()

Bx_reinput,By_reinput,Bz_reinput = Sdp.extract_Bxyz(B_NV_test, metadata, theta, phi, eps = 0.001, nr_expansions=1, dz=dz, apply_hann=False)

Bx_reinput = Bx_reinput-np.mean(Bx_reinput)
By_reinput = By_reinput-np.mean(By_reinput)
Bz_reinput = Bz_reinput-np.mean(Bz_reinput)


pSd.PlotData(Bx,metadata,datalabel='Bx')
plt.show()
pSd.PlotData(Bx_reinput,metadata,datalabel='Bx_reinput')
plt.show()

pSd.PlotData(By,metadata,datalabel='By')
plt.show()
pSd.PlotData(By_reinput,metadata,datalabel='By_reinput')
plt.show()

pSd.PlotData(Bz,metadata,datalabel='Bz')
plt.show()
pSd.PlotData(Bz_reinput,metadata,datalabel='Bz_reinput')
plt.show()



In [ ]:
dz=100.00e-9
theta = 90
phi = 45
#theta = 90
#phi = 35.25

data_x = np.ones(np.shape(Bx))
data_y = np.ones(np.shape(By))
#data_y = np.outer(np.linspace(-1,1,np.shape(By)[0]),np.ones(np.shape(By)[1]))
data_z = np.zeros(np.shape(Bz))

px_x = 250
px_y = 250

data_x = np.ones((px_x,px_y))
data_x = np.outer(np.ones((px_x,px_y)[1]),np.sin(4*np.pi*np.linspace(-1,1,(px_x,px_y)[0])))
data_x = np.outer(np.sin(20*np.pi*np.linspace(-1,1,(px_x,px_y)[0])),np.sin(20*np.pi*np.linspace(-1,1,(px_x,px_y)[0])))
data_y = np.zeros((px_x,px_y))
data_y = np.outer(np.sin(40*np.pi*np.linspace(-1,1,(px_x,px_y)[0])),np.sin(40*np.pi*np.linspace(-1,1,(px_x,px_y)[0])))
#data_y = np.outer(np.linspace(-1,1,(px_x,px_y)[0]),np.ones((px_x,px_y)[1]))
data_z = np.zeros((px_x,px_y))
#data_z = np.outer(np.linspace(-1,1,(px_x,px_y)[0]),np.ones((px_x,px_y)[1]))
data_z = np.outer(np.sin(60*np.pi*np.linspace(-1,1,(px_x,px_y)[0])),np.sin(60*np.pi*np.linspace(-1,1,(px_x,px_y)[0])))

pSd.PlotData(data_x, metadata,datalabel='data_x')
plt.show()
pSd.PlotData(data_y, metadata,datalabel='data_y')
plt.show()
pSd.PlotData(data_z, metadata,datalabel='data_z')
plt.show()




ex, ey, ez = Sdp.get_NV_unit_vectors(theta,phi)
print(ex,ey,ez)
B_NV_test = (data_x*ex + data_y*ey  + data_z*ez)

print(np.mean(B_NV_test),B_NV_test)
pSd.PlotData(B_NV_test, metadata)
plt.show()

print(np.shape(data))

Bx, By, Bz = Sdp.extract_Bxyz(B_NV_test, metadata, theta, phi, 
                              eps = 0.001, nr_expansions=1, dz=dz, apply_hann=False)


pSd.PlotData(Bx, metadata,datalabel='Bx')
plt.show()
pSd.PlotData(By, metadata,datalabel='By')
plt.show()
pSd.PlotData(Bz, metadata,datalabel='Bz')
plt.show()

print(np.mean(data_x),np.mean(data_y),np.mean(data_z))
print(np.mean(Bx),np.mean(By),np.mean(Bz))
print(Bx)
print(By)
print(Bz)

angles = np.zeros((np.shape(Bx)[0],np.shape(Bx)[1]))
B_IP_proj = np.zeros((np.shape(Bx)[0],np.shape(Bx)[1]))
angles_x_unitcircle = np.zeros((np.shape(Bx)[0],np.shape(Bx)[1]))
angles_y_unitcircle = np.zeros((np.shape(By)[0],np.shape(By)[1]))

for i in range(np.shape(Bx)[0]):
    for j in range(np.shape(Bx)[1]):
        angle_B_IP_rad = np.arctan2(By[i,j],Bx[i,j])
        angle_B_IP_deg = angle_B_IP_rad*360/(2*np.pi)
        angles[i,j] = angle_B_IP_deg 
        B_IP_proj[i,j] = np.sqrt(Bx[i,j]**2+By[i,j]**2)
        angles_x_unitcircle[i,j] = np.sin(angle_B_IP_rad)
        angles_y_unitcircle[i,j] = np.cos(angle_B_IP_rad)
        

#x = np.linspace(0,np.shape(Bx)[0]-1,np.shape(Bx)[0])
#y = np.linspace(0,np.shape(Bx)[1]-1,np.shape(Bx)[1])

Nr_arrows_per_line = 50 #268/4 = 67 bc view_as_blocks has to fit in perfectly and size of Bx of 269 is a prime number
data_cutoff = 250
blocksize = int(data_cutoff/Nr_arrows_per_line) #blocksize has to be integer before conversion

x = np.linspace(0,data_cutoff,Nr_arrows_per_line)
y = np.linspace(0,data_cutoff,Nr_arrows_per_line)

##### prepare Bxy to be plotted as arrows ####

Bx_blocks = view_as_blocks(Bx[:data_cutoff,:data_cutoff],block_shape=(blocksize,blocksize))
By_blocks = view_as_blocks(By[:data_cutoff,:data_cutoff],block_shape=(blocksize,blocksize))

Bx_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))
By_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))

for i in range(np.shape(Bx_blocks)[0]):
    for j in range(np.shape(Bx_blocks)[1]):
        Bx_reduced_avg[i,j] = np.mean(Bx_blocks[i,j,:,:])

for i in range(np.shape(By_blocks)[0]):
    for j in range(np.shape(By_blocks)[1]):
        By_reduced_avg[i,j] = np.mean(By_blocks[i,j,:,:])

#print(np.shape(Bx_reduced_avg),np.shape(By_reduced_avg),Bx_reduced_avg,By_reduced_avg)
#print(np.shape(Bx),np.shape(x),np.shape(y),np.shape(Bx_reshaped),np.shape(By_reshaped))
#print(np.shape(Bx),x,y,np.shape(Bx_reshaped),np.shape(By_reshaped))

###############################################

fig, ax = plt.subplots()
#fig = plt.figure(figsize=(20,20))
im = ax.imshow(angles,cmap = 'hsv')#,vmin=-180,vmax=180)
#arrows = ax.quiver(x,y,Bx_reduced_avg,By_reduced_avg)
cb=fig.colorbar(im)
cb.set_label('B_IP Angle')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)



######## try to change saturation with B_IP strength ########

#plt.show()
fig.canvas.draw()
orig_im_RGBarray = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
orig_image = orig_im_RGBarray.reshape(fig.canvas.get_width_height()[::-1] + (3,))

angle_im = cv2.cvtColor(orig_image,cv2.COLOR_BGR2HSV_FULL)
h=angle_im[:,:,0].copy()
s=angle_im[:,:,1].copy()
#print(np.shape(s))
v=angle_im[:,:,2].copy()
s=s
#v=v+0.001
angle_im=np.dstack((h,s,v))
angle_im=cv2.cvtColor(angle_im,cv2.COLOR_HSV2RGB_FULL)

#plt.imshow(angle_im)

##############################################################


#angles_x_blocks = view_as_blocks(angles_x_unitcircle[:data_cutoff,:data_cutoff],block_shape=(blocksize,blocksize))
#angles_y_blocks = view_as_blocks(angles_y_unitcircle[:data_cutoff,:data_cutoff],block_shape=(blocksize,blocksize))
angles_blocks = view_as_blocks(angles[:data_cutoff,:data_cutoff],block_shape=(blocksize,blocksize))

#angles_x_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))
#angles_y_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))
angles_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))

#for i in range(np.shape(angles_x_blocks)[0]):
#    for j in range(np.shape(angles_x_blocks)[1]):
#        angles_x_reduced_avg[i,j] = np.mean(angles_x_blocks[i,j,:,:])

#for i in range(np.shape(angles_y_blocks)[0]):
#    for j in range(np.shape(angles_y_blocks)[1]):
#        angles_y_reduced_avg[i,j] = np.mean(angles_y_blocks[i,j,:,:])
        
for i in range(np.shape(angles_blocks)[0]):
    for j in range(np.shape(angles_blocks)[1]):
        angles_reduced_avg[i,j] = np.mean(angles_blocks[i,j,:,:])
        
#print(angles_x_reduced_avg,angles_y_reduced_avg)   

u = np.zeros_like(angles_reduced_avg)+np.sqrt(0.5)
v = np.zeros_like(angles_reduced_avg)+np.sqrt(0.5)

data_angles = np.zeros((np.shape(data_x)[0],np.shape(data_x)[1]))
data_IP_proj = np.zeros((np.shape(data_x)[0],np.shape(data_x)[1]))
data_angles_blocks = view_as_blocks(data_angles[:data_cutoff,:data_cutoff],block_shape=(blocksize,blocksize))
data_angles_reduced_avg = np.zeros((Nr_arrows_per_line,Nr_arrows_per_line))

for i in range(np.shape(data_x)[0]):
    for j in range(np.shape(data_x)[1]):
        angle_data_IP_rad = np.arctan2(data_y[i,j],data_x[i,j])
        angle_data_IP_deg = angle_data_IP_rad*360/(2*np.pi)
        data_angles[i,j] = angle_data_IP_deg 
        data_IP_proj[i,j] = np.sqrt(data_x[i,j]**2+data_y[i,j]**2)

for i in range(np.shape(data_angles_blocks)[0]):
    for j in range(np.shape(data_angles_blocks)[1]):
        data_angles_reduced_avg[i,j] = np.mean(data_angles_blocks[i,j,:,:])

fig, ax = plt.subplots()
im = ax.imshow(data_IP_proj[:data_cutoff,:data_cutoff],cmap = 'summer',
               extent = (0,data_cutoff,data_cutoff,0))
#arrows = ax.quiver(x,y,angles_x_reduced_avg,angles_y_reduced_avg,scale=40,angles=angles_reduced_avg,pivot='mid') 
arrows = ax.quiver(x,y,u,v,scale=50,angles=data_angles_reduced_avg,pivot='mid') 
# angles_xy not used anymore, only for length of arrows. directions are directly set by angles
cb=fig.colorbar(im)
cb.set_label('data_IP')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)    

fig, ax = plt.subplots()
im = ax.imshow(B_IP_proj[:data_cutoff,:data_cutoff],cmap = 'summer',
               extent = (0,data_cutoff,data_cutoff,0))
arrows = ax.quiver(x,y,u,v,scale=50,angles=angles_reduced_avg,pivot='mid') 
# angles_xy not used anymore, only for length of arrows. directions are directly set by angles
cb=fig.colorbar(im)
cb.set_label('B_IP')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
        
fig, ax = plt.subplots()
im = ax.imshow(Bz,cmap = 'summer')
cb=fig.colorbar(im)
cb.set_label('B_z')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)


#proj_ang_rad = proj_ang*2*np.pi/360
#B_IP = np.sqrt(Bx**2+By**2)
#angle_B_IP = np.arctan2(By,Bx)
#angle_B_proj_B_IP = proj_ang_rad-angle_B_IP
#B_proj = np.sin(angle_B_proj_B_IP)*B_IP
#B_proj = np.sin(angle_B_proj_B_IP)*B_IP
#print(proj_ang_rad, B_IP, angle_B_IP, angle_B_proj_B_IP)


In [ ]:
basepath = '/home/julisko/Seafile/My Library/SNVM/20230227_QZabre_Visit/Visit_Feb_2023'
folder = '/2023-02-27/odmr_fast' #Fe2O3 Stripes
folderday = '/2023-02-27-15-02-13-odmr_fast'
file,metadata = pSd.ReadSNVMfile(basepath+folder+folderday)
data = pSd.ExtractData(file,datatype = 'odmr:mTesla')
data = pSd.AdjustDataToQS3(data)

def gaussian(x, mu, sig):
    return 1./(np.sqrt(2.*np.pi)*sig)*np.exp(-np.power((x - mu)/sig, 2.)/2)

dz=100.00e-9
#theta = 144.75
#phi = 270

#theta = 360*np.random.random()
#phi = 360*np.random.random()

theta = 0
phi = 0

px_x = metadata['resolution'][1]
px_y = metadata['resolution'][0]

#px_x=50
#px_y=50

actualshape = (20e-6,20e-6)

plane_range = 1

data_x_xrange = np.linspace(-1,1,px_x)*plane_range
data_x_yrange = np.linspace(-1,1,px_y)*plane_range
data_y_xrange = np.linspace(-1,1,px_x)*plane_range
data_y_yrange = np.linspace(-1,1,px_y)*plane_range
data_z_xrange = np.linspace(-1,1,px_x)*plane_range
data_z_yrange = np.linspace(-1,1,px_y)*plane_range

data_x = np.outer(np.sin(2*np.pi*data_x_yrange),np.sin(4*np.pi*data_x_xrange))*0
data_y = np.outer(np.sin(4*np.pi*data_y_yrange),np.sin(6*np.pi*data_y_xrange))*0
data_z = np.outer(np.sin(6*np.pi*data_z_yrange),np.sin(8*np.pi*data_z_xrange))


###### arbitrary vector field v_xyz = v_xyz(x,y)#########

x_grid2D = np.meshgrid(data_x_xrange,data_x_yrange)
y_grid2D = np.meshgrid(data_y_xrange,data_y_yrange)
z_grid2D = np.meshgrid(data_z_xrange,data_z_yrange)

def vx(x,y):
    #return(-y/(x*x+y*y))
    return -2*x
def vy(x,y):
    #return(x/(x*x+y*y))
    return 6*np.ones_like(x)
def vz(x,y):
    #return(0*x+0*y)
    return 0*np.ones_like(x)
    
#data_x = vx(x_grid2D[0],x_grid2D[1])
#data_y = vy(y_grid2D[0],y_grid2D[1])
#data_z = vz(z_grid2D[0],z_grid2D[1])

## plot 2D arrows of vector field v in xy-plane
fig, ax = plt.subplots()
plt.quiver(x_grid2D[0],x_grid2D[1],data_x,data_y)
plt.show()

#########################################################

print(np.shape(data_x),np.shape(data_y),np.shape(data_z))

datastack = [data_x,data_y,data_z]

data_x_ft = np.fft.fftshift(np.fft.fft2(datastack[0]))
data_y_ft = np.fft.fftshift(np.fft.fft2(datastack[1]))
data_z_ft = np.fft.fftshift(np.fft.fft2(datastack[2]))

print(np.max(np.abs(data_x_ft)),np.max(np.abs(data_y_ft)),np.min(np.abs(data_z_ft)))

pSd.PlotData((np.real(data_x_ft)),metadata,datalabel='data x FFT')
plt.show()
pSd.PlotData((np.real(data_y_ft)),metadata,datalabel='data y FFT')
plt.show()
pSd.PlotData((np.real(data_z_ft)),metadata,datalabel='data z FFT')
plt.show()

B_NV_sim, Bx_sim, By_sim, Bz_sim = Sdp.Bxyz_toBNV_toBxyz(datastack, metadata, theta, phi, eps=0.001, 
                      nr_expansions=1, dz=0, apply_hann=False, num_output=True)#, actualshape=actualshape)

B_NV_sim_FFT = np.fft.fftshift(np.fft.fft2(B_NV_sim))
pSd.PlotData((np.real(B_NV_sim_FFT)),metadata,datalabel='B_NV sim FFT')
plt.show()

print(np.shape(Bx_sim),np.shape(By_sim),np.shape(Bz_sim))

print('-----------------------------------------------------------------------------------------')


B_real_x, B_real_y, B_real_z = Sdp.extract_Bxyz(data, metadata, theta, phi, 
                                                eps = 0.001, nr_expansions=1, dz=1e-9, apply_hann=False)

B_real_stack = [B_real_x,B_real_y,B_real_z]

B_NV_real, Bx_real, By_real, Bz_real = Sdp.Bxyz_toBNV_toBxyz(B_real_stack, metadata, theta, phi, eps=0.001, 
                      nr_expansions=1, dz=0, apply_hann=False, num_output=True)


# --------------- Additional or deprecated code -----------------------

In [ ]:
dz=100.00e-9
theta = 90
phi = 0
#theta = 90
#phi = 35.25

data_x = np.ones(np.shape(Bx))
data_y = np.zeros(np.shape(By))
#data_y = np.outer(np.linspace(-1,1,np.shape(By)[0]),np.ones(np.shape(By)[1]))
data_z = np.zeros(np.shape(Bz))

theta_rad = theta*2*np.pi/360
phi_rad = phi*2*np.pi/360
ex, ey, ez = Sdp.get_NV_unit_vectors(theta_rad,phi_rad)
print(ex,ey,ez)
B_NV_test = (data_x*ex + data_y*ey  + data_z*ez)

print(B_NV_test)
pSd.PlotData(B_NV_test, metadata)
plt.show()

Bft = np.fft.fftshift(np.fft.fft2(B_NV_test))
pSd.PlotData(np.real(Bft), metadata,maxval=1)
plt.show()


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='polar')

rho = np.linspace(0,1,30) # Radius of 1, distance from center to outer edge
phi = np.linspace(0, np.pi*2.,100 ) # in radians, one full circle

RHO, PHI = np.meshgrid(rho,phi) # get every combination of rho and phi

h = (PHI-PHI.min()) / (PHI.max()-PHI.min()) # use angle to determine hue, normalized from 0-1
h = np.flip(h)        
s = RHO               # saturation is set as a function of radias
v = np.ones_like(RHO) # value is constant
#v = np.ones_like(RHO)*np.outer(np.ones(np.shape(RHO)[0]),np.linspace(0,1,np.shape(RHO)[1])) # value is constant
print(np.shape(RHO))
# convert the np arrays to lists. This actually speeds up the colorsys call
h,s,v = h.flatten().tolist(), s.flatten().tolist(), v.flatten().tolist()
c = [colorsys.hsv_to_rgb(*x) for x in zip(h,s,v)]
c = np.array(c)

#cmap_name = 'twilight_shifted'
cmap_name ='hsv'
cmap = plt.get_cmap(cmap_name)
norm = mpl.colors.Normalize()
scalarMap = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
#c = scalarMap.to_rgba(np.outer(np.ones(np.shape(RHO)[0]), np.linspace(0,1,np.shape(RHO)[1])))
c = scalarMap.to_rgba(np.linspace(0,1,np.shape(RHO)[1]*np.shape(RHO)[0]))
#print(c)

alpha = np.outer(np.ones(np.shape(RHO)[0]),np.linspace(1,0,np.shape(RHO)[1]))
ax.scatter(PHI, RHO, c=c )#  ,alpha=alpha)
_ = ax.axis('off')

In [ ]:
#plot Bx, By, Bz
pSd.PlotData(B_proj,metadata)
plt.show()
pSd.PlotData(Bx,metadata)
plt.show()
pSd.PlotData(By,metadata)
plt.show()
pSd.PlotData(Bz,metadata)
plt.show()

In [ ]:
test = np.linspace(1,6,6).reshape(2,3)
test = 2*[test]
test = 2*[test]
test = np.block(test)
print(test)

In [ ]:
# colorwheel plot of angle yielding maximal value

dz=100e-9
theta = 144.75
phi = 270

proj_angles = np.arange(0,360,10)

B_proj_all = np.zeros((len(proj_angles),np.shape(Bx)[0],np.shape(Bx)[1])) 

for i in range(len(proj_angles)):
    proj_ang = proj_angles[i]
    B_proj = Sdp.extract_BIP_Projected_onto_Angle(data, metadata, theta, phi, proj_ang, dz=dz)
    B_proj_all[i,:,:] = B_proj

ang_max = np.zeros((np.shape(Bx)[0],np.shape(Bx)[1]))

for i in range(np.shape(Bx)[0]):
    for j in range(np.shape(Bx)[1]):
        maxval_index = B_proj_all[:,i,j].argmax() # argmax returns index of maximum value
        ang_max[i,j] = proj_angles[maxval_index]
        
fig, ax = plt.subplots()
im = plt.imshow(ang_max)
cb=fig.colorbar(im)
cb.set_label('Angle of maximum value')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

In [ ]:
cm = 'twilight_shifted'
fig8, ax8 = plt.subplots(subplot_kw=dict(projection='polar'))
ang_rav_cut = np.linspace(0,10000000,100000)
#shift bins:
x = ang_rav_cut.copy()
#x = np.where(x<-85,x+180,x)
bins = 18
b,c = np.histogram(x,bins=bins)
ca = c[0:-1] + np.diff(c)[0]/2+90
cmap = plt.cm.mpl.colormaps[cm]
colors = cmap((ca)/180)
#ax8 = plt.subplot(111, polar=True)
ax8.bar(np.deg2rad(ca)+np.pi,b,color=colors,width=np.pi/bins)
ax8.bar(np.deg2rad(ca),b[:],color=colors,width=np.pi/bins)
ax8.axes.yaxis.set_visible(False)

In [ ]:
fig, ax = plt.subplots()
#fig = plt.figure(figsize=(20,20))
im = ax.imshow(Bx,cmap = 'bwr')#,vmin=-180,vmax=180)
#arrows = ax.quiver(x,y,Bx_reduced_avg,By_reduced_avg)
cb=fig.colorbar(im)
cb.set_label('B_x')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

fig, ax = plt.subplots()
#fig = plt.figure(figsize=(20,20))
im = ax.imshow(By,cmap = 'bwr')#,vmin=-180,vmax=180)
#arrows = ax.quiver(x,y,Bx_reduced_avg,By_reduced_avg)
cb=fig.colorbar(im)
cb.set_label('B_y')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

fig, ax = plt.subplots()
#fig = plt.figure(figsize=(20,20))
im = ax.imshow(Bz,cmap = 'bwr')#,vmin=-180,vmax=180)
#arrows = ax.quiver(x,y,Bx_reduced_avg,By_reduced_avg)
cb=fig.colorbar(im)
cb.set_label('B_z')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)